In [1]:
import sagemaker
from sagemaker.s3 import S3Uploader
from datasets import load_dataset
import os

# --- 1. Define the Hugging Face Dataset and Load It ---
# We will use the 'yahma/alpaca-cleaned' dataset.
# It is already split into 'train' and formatted for instruction tuning.
DATASET_ID = "yahma/alpaca-cleaned"
print(f"Loading dataset: {DATASET_ID}")

# Load the dataset directly from the Hugging Face Hub
# We explicitly load the 'train' split
dataset = load_dataset(DATASET_ID, split="train")

# --- 2. Format the Dataset for the SFTTrainer ---
# The original Alpaca dataset has 'instruction', 'input', and 'output' columns.
# We must combine these into a single 'text' column for the SFTTrainer.

def formatting_function(example):
    """
    Formats the Alpaca instruction/input/output into a single 'text' string
    in the Alpaca template, which the SFTTrainer will use.
    """
    instruction = example['instruction']
    output = example['output']
    
    # Check if there is an additional 'input' field (for context)
    if example.get("input"):
        template = f"""### Instruction:
{instruction}

### Input:
{example['input']}

### Response:
{output}"""
    else:
        template = f"""### Instruction:
{instruction}

### Response:
{output}"""
    
    return {"text": template}

# Apply the formatting function to the entire dataset
dataset = dataset.map(formatting_function, remove_columns=['instruction', 'input', 'output'])
dataset = dataset.select_columns(['text']) # Keep only the required 'text' column

print("\nFormatted Dataset Example:")
print(dataset[0]['text'])

# --- 3. Save Dataset Locally and Upload to S3 ---
local_data_path = 'alpaca_qlora_data'
if not os.path.exists(local_data_path):
    os.makedirs(local_data_path)

# Save the dataset to a local folder in the correct format
dataset.save_to_disk(local_data_path)
print(f"\nDataset saved locally to: {local_data_path}")

# Setup S3 paths
sess = sagemaker.Session()
bucket = sess.default_bucket()
s3_prefix = 'qlora-alpaca-validation'
s3_data_uri = f"s3://{bucket}/{s3_prefix}/data/train"

# Upload the local dataset directory to S3
S3Uploader.upload(
    local_path=local_data_path, 
    desired_s3_uri=s3_data_uri, 
    sagemaker_session=sess
)

print(f"\nTraining data successfully uploaded to S3: {s3_data_uri}")

# --- Set the Input for the Training Job ---
inputs = {
    'training': s3_data_uri 
}

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
Loading dataset: yahma/alpaca-cleaned


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:14                                                                                   │
│                                                                                                  │
│   11                                                                                             │
│   12 # Load the dataset directly from the Hugging Face Hub                                       │
│   13 # We explicitly load the 'train' split                                                      │
│ ❱ 14 dataset = load_dataset(DATASET_ID, split="train")                                           │
│   15                                                                                             │
│   16 # --- 2. Format the Dataset for the SFTTrainer ---                                          │
│   17 # The original Alpaca dataset has 'instruction', 'input', and 'output' columns.             │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/datasets/load.py:1704 in load_dataset                    │
│                                                                                                  │
│   1701 │   ignore_verifications = ignore_verifications or save_infos                             │
│   1702 │                                                                                         │
│   1703 │   # Create a dataset builder                                                            │
│ ❱ 1704 │   builder_instance = load_dataset_builder(                                              │
│   1705 │   │   path=path,                                                                        │
│   1706 │   │   name=name,                                                                        │
│   1707 │   │   data_dir=data_dir,                                                                │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/datasets/load.py:1530 in load_dataset_builder            │
│                                                                                                  │
│   1527 │   if use_auth_token is not None:                                                        │
│   1528 │   │   download_config = download_config.copy() if download_config else DownloadConfig(  │
│   1529 │   │   download_config.use_auth_token = use_auth_token                                   │
│ ❱ 1530 │   dataset_module = dataset_module_factory(                                              │
│   1531 │   │   path,                                                                             │
│   1532 │   │   revision=revision,                                                                │
│   1533 │   │   download_config=download_config,                                                  │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/datasets/load.py:1282 in dataset_module_factory          │
│                                                                                                  │
│   1279 │   │   │   │   │   │   f"Couldn't find a dataset script at {relative_to_absolute_path(c  │
│   1280 │   │   │   │   │   │   f"Couldn't find '{path}' on the Hugging Face Hub either: {type(e  │
│   1281 │   │   │   │   │   ) from None                                                           │
│ ❱ 1282 │   │   │   │   raise e1 from None                                                        │
│   1283 │   else:                                                                                 │
│   1284 │   │   raise FileNotFoundError(                                                          │
│   1285 │   │   │   f"Couldn't find a dataset script at {rel

In [ ]:
import sagemaker
from sagemaker.huggingface import HuggingFace

# --- 1. Define Sagemaker Session and Execution Role ---
sess = sagemaker.Session()
role = sagemaker.get_execution_role()

# --- 2. Configuration for the Training Job ---
instance_type = 'ml.g5.2xlarge'  
instance_count = 1
# *** Using the LATEST SUPPORTED BASE IMAGE ***
transformers_version = '4.46.1'  
pytorch_version = '2.3.0'        # Corrected: Latest version supported by your SDK
py_version = 'py311'             # Corrected: Python 3.11 is typically used with PyTorch 2.3.0

# Hyperparameters (no change)
hyperparameters = {
    'model_id': 'mistralai/Mistral-7B-v0.1', 
    'output_dir': '/opt/ml/model',         
    'epochs': 3,                           
    'per_device_train_batch_size': 4,
    'gradient_accumulation_steps': 2,      
    'learning_rate': 2e-4,                 
    'max_seq_length': 1024,
    'packing': True,                       
    'lora_r': 64,                          
    'lora_alpha': 16,                      
}

# --- 3. Create the HuggingFace Estimator ---
huggingface_estimator = HuggingFace(
    entry_point='train.py',       
    source_dir='scripts',         
    instance_type=instance_type,
    instance_count=instance_count,
    role=role,
    transformers_version=transformers_version,
    pytorch_version=pytorch_version,
    py_version=py_version,
    hyperparameters=hyperparameters,
    max_run=36000, 
)

# --- 4. Launch the Training Job ---
print(f"Launching QLoRA training job on {instance_type}...")

# Use the 'inputs' variable created from the S3 upload
huggingface_estimator.fit(
    inputs=inputs, 
    wait=False  # Keep wait=False for remote monitoring
)

print("\nTraining Job launched! Monitor progress in the SageMaker Console.")
print(f"Job Name: {huggingface_estimator.latest_training_job.job_name}")